## Collecting the dataset from OpenStreetMap

## Libaries installation

In [0]:
import json
import requests
import os
from shutil import copy

### Links about solving problems which appear on Colab


*  ```fatal error: sqlite3.h: No such file or directory``` – https://github.com/mapbox/tippecanoe/issues/601
*   https://github.com/mapbox/tippecanoe/issues/619



In [0]:
!sudo apt-get install build-essential libsqlite3-dev zlib1g-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
Suggested packages:
  sqlite3-doc
The following NEW packages will be installed:
  libsqlite3-dev
0 upgraded, 1 newly installed, 0 to remove and 8 not upgraded.
Need to get 632 kB of archives.
After this operation, 2,124 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsqlite3-dev amd64 3.22.0-1 [632 kB]
Fetched 632 kB in 0s (5,099 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> l

In [0]:
!git clone https://github.com/mapbox/tippecanoe.git
os.chdir("tippecanoe")
!make -j
!make install

os.chdir("/content")
!ls

Cloning into 'tippecanoe'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 13175 (delta 4), reused 8 (delta 1), pack-reused 13155
Receiving objects: 100% (13175/13175), 29.73 MiB | 18.99 MiB/s, done.
Resolving deltas: 100% (8674/8674), done.
g++ -MMD  -I/usr/local/include -I. -g -Wall -Wshadow -Wsign-compare -Wextra -Wunreachable-code -Wuninitialized -Wshadow -O3 -DNDEBUG  -std=c++11 -c -o geojson.o geojson.cpp
cc -MMD  -I/usr/local/include -I. -g -Wall -Wshadow -Wsign-compare -Wextra -Wunreachable-code -Wuninitialized -Wshadow -O3 -DNDEBUG  -c -o jsonpull/jsonpull.o jsonpull/jsonpull.c
g++ -MMD  -I/usr/local/include -I. -g -Wall -Wshadow -Wsign-compare -Wextra -Wunreachable-code -Wuninitialized -Wshadow -O3 -DNDEBUG  -std=c++11 -c -o tile.o tile.cpp
g++ -MMD  -I/usr/local/include -I. -g -Wall -Wshadow -Wsign-compare -Wextra -Wunreachable-code -Wuninitialized -Wshadow -O3 -DNDEBUG  -st

In [0]:
!git clone https://github.com/NickElixir/label-maker.git
os.chdir("label-maker")
!ls
!python setup.py install
os.chdir("/content")
!ls

Cloning into 'label-maker'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 1348 (delta 22), reused 26 (delta 11), pack-reused 1296
Receiving objects: 100% (1348/1348), 11.14 MiB | 9.71 MiB/s, done.
Resolving deltas: 100% (781/781), done.
config_russia.json  label_maker  MANIFEST.in  requirements-dev.txt  setup.py
docs		    LICENSE.txt  README.md    requirements.txt
running install
running bdist_egg
running egg_info
creating label_maker.egg-info
writing label_maker.egg-info/PKG-INFO
writing dependency_links to label_maker.egg-info/dependency_links.txt
writing entry points to label_maker.egg-info/entry_points.txt
writing requirements to label_maker.egg-info/requires.txt
writing top-level names to label_maker.egg-info/top_level.txt
writing manifest file 'label_maker.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'label_maker.egg-info/SOURCES.txt'
inst

## Mounting Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')
workspace_folder = '/content/drive/My Drive/Datasets/osm dataset/'
os.chdir(workspace_folder)
!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
 config.json  'dataset(colab).ipynb'   tags.txt
 config.txt    russia_airports.txt     token.txt


## Loading configuration

In [0]:
config = open("config.txt", "r")
config_list = config.readlines()

country = config_list[0][:-1].lower() #drop off \n and capital letters
min_zoom_level = 16
max_zoom_level = 18
dA = float(config_list[3])
ml_type = str(config_list[4])

config.close()
print(country, min_zoom_level, max_zoom_level, dA, ml_type)

russia 16 18 0.01 segmentation


## Loading tags 

In [0]:
tags_file = open("tags.txt", "r")
tags = []
for line in tags_file:
    tags.append(line.replace('\n', ""))
tags = sorted(tags)
print(tags)

aeroways_file = open("values/aeroway.txt", "r")
aeroways = []
for line in aeroways_file:
    aeroways.append(line.replace('\n', ""))
aeroways = sorted(aeroways)
print(aeroways)

highways_file = open("values/highway.txt", "r")
highways = []
for line in highways_file:
    highways.append(line.replace('\n', ""))
highways = sorted(highways)
print(highways)

landuses_file = open("values/landuse.txt", "r")
landuses = []
for line in landuses_file:
    landuses.append(line.replace('\n', ""))
landuses = sorted(landuses)
print(landuses)

naturals_file = open("values/natural.txt", "r")
naturals = []
for line in naturals_file:
    naturals.append(line.replace('\n', ""))
naturals = sorted(naturals)
print(naturals)

waterways_file = open("values/waterway.txt", "r")
waterways = []
for line in waterways_file:
    waterways.append(line.replace('\n', ""))
waterways = sorted(waterways)
print(waterways)

['aeroway', 'buildings', 'highway', 'landuse', 'natural', 'offices', 'waterway']
['aerodrom', 'apron', 'runway', 'taxiway']
['motorway', 'primary', 'residential', 'road', 'secondary', 'tertiary', 'trunk']
['farmland', 'forest', 'grass', 'greenfield', 'residential']
['grassland', 'heath', 'scrub', 'tree', 'wood']
['drystream', 'river', 'riverbank', 'stream']


## Initializing filters for configurations

In [0]:
filters = []
for tag in tags:
    filters.append({ "name": tag.title(), "filter": ["has", tag] })
for aeroway in aeroways:
    filters.append({"name": aeroway.title(), "filter": ["==", "aeroway", aeroway]})
for highway in highways:
    filters.append({"name": highway.title(), "filter": ["==", "highway", highway]})
for landuse in landuses:
    filters.append({"name": landuse.title(), "filter": ["==", "landuse", landuse]})
for waterway in waterways:
    filters.append({"name": waterway.title(), "filter": ["==", "waterway", waterway]})

## OpenStreetMap token

In [0]:
token = open("token.txt", "r")
for i in token:
    token = i
print(token)

pk.eyJ1IjoiemVyb2x5bXAiLCJhIjoiY2p2NWJ2aW9rMDFyNTN5bzZlMTY3NmFucSJ9.SrPXLPkFpd22vCnYsSvxoA


## Downloading  QA tiles

You should download QA tiles before running the loop code. Use this command after creating config.json in next steps.

In [0]:
mbtiles_folder = '"/content/drive/My Drive/Datasets/osm dataset"'
if not os.path.exists(workspace_folder + country + ".mbtiles"):
    os.chdir(workspace_folder)
    !label-maker download --config config.json --dest $mbtiles_folder
os.chdir(workspace_folder)
!ls

Saving QA tiles to /content/drive/My Drive/Datasets/osm dataset/russia.mbtiles
   100%      5.5 GiB      21.6 MiB/s            0:00:00 ETA    
 config.gdoc   config.txt	       russia_airports.txt   tags.txt	 values
 config.json  'dataset(colab).ipynb'   russia.mbtiles	     token.txt


## Choose your airport ID

In [0]:
start = 3
finish = start + 1

In [0]:
airports = open(country + "_airports.txt", "r")
i = 0

for airport in airports:
    print(i)
    if i >= start and i < finish:
        airport = airport.replace("\n", "").split(";")
        print(airport)
        
        name = airport[0]
        lat_min = float(airport[1])
        lat_max = float(airport[2])
        long_min = float(airport[3])
        long_max = float(airport[4])
            
            
        for current_level in range(int(min_zoom_level), int(max_zoom_level) + 1):
            dir_name = name.replace(" ", "_") + "_" + str(current_level)
            print(dir_name)
            
            config_dict = {
                "country": country,
                "bounding_box": [long_min - dA, lat_min - dA, long_max + dA, lat_max + dA],
                "zoom": current_level,
                "classes": filters,
                "imagery": "http://a.tiles.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}.jpg?access_token=" + token,
                "background_ratio": 1,
                "ml_type": ml_type
            }
            with open("config.json", "w") as config_file:
                json.dump(config_dict, config_file)
            
            !label-maker labels --dest $dir_name --config config.json --mbtiles $mbtiles_folder
            #!label-maker preview -n 10 --dest $dir_name --config config.json
            !label-maker images --dest $dir_name --config config.json
            !label-maker package --dest $dir_name --config config.json
                
    i += 1
airports.close()

0
1
2
3
['Khabarovsk-Novy Airport', '48.508723', '48.5474307', '135.162754', '135.2157047']
Khabarovsk-Novy_Airport_16
/content/drive/My Drive/Datasets/osm dataset/russia.mbtiles
Retiling QA Tiles to zoom level 16 (takes a bit)
2913 features, 227623 bytes of geometry, 15061 bytes of separate metadata, 103530 bytes of string pool
  99.9%  16/57392/22633  
Determining labels for each tile
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 15 255 at 15 255
The operation 'GEOSIntersection_r' could not be performed. Likely cause is invalidity of the geometry <shapely.geometry.polygon.Polygon object at 0x7f5b4b4bd8d0> skipping
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 16 245 at 16 245
The operation 'GEOSIntersection_r' could not be performed. Likely cause is invalidity of the geometry <shapely.geometry.multipolygon.MultiPolygon object at 0x7f5b4b4bb278> skipping
TopologyException: Input geom 0 is invalid: Self-intersection at o

## Links

using requests – https://www.geeksforgeeks.org/get-post-requests-using-python/

### Delete label-maker repository

In [0]:
os.chdir("/content")
!ls
!rm -r /content/label-maker

drive  label-maker  sample_data  tippecanoe
